In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [3]:
 mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
       
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
    
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
    
    
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /=255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))
 

In [8]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation= 'softmax')
                            ])

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
Num_EPOCHS = 5

model.fit(train_data,epochs=Num_EPOCHS,validation_data=(validation_inputs,validation_targets),validation_steps=1,verbose=2)

Epoch 1/5
540/540 - 17s - loss: 0.3182 - accuracy: 0.9083 - val_loss: 0.1609 - val_accuracy: 0.9533
Epoch 2/5
540/540 - 11s - loss: 0.1320 - accuracy: 0.9612 - val_loss: 0.1176 - val_accuracy: 0.9650
Epoch 3/5
540/540 - 12s - loss: 0.0917 - accuracy: 0.9728 - val_loss: 0.0880 - val_accuracy: 0.9750
Epoch 4/5
540/540 - 10s - loss: 0.0715 - accuracy: 0.9787 - val_loss: 0.0726 - val_accuracy: 0.9788
Epoch 5/5
540/540 - 10s - loss: 0.0580 - accuracy: 0.9823 - val_loss: 0.0577 - val_accuracy: 0.9832


In [11]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step - loss: 0.0828 - accuracy: 0.9750


In [12]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.50%
